# AE on proteins in SA representation

In [ ]:
import glob
import os
from collections import Counter
import string
from keras import Input
from keras.layers import Dense, Lambda, Conv1D
import keras.backend as K
from keras.models import Model
from keras.objectives import binary_crossentropy, mse
import os
import random
import numpy as np
import NotebookLoader
from keras.optimizers import RMSprop, Adam
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
from tempfile import TemporaryFile
import csv

In [ ]:
import Preprocessing as pre

### Configs

In [ ]:
lengths = {}
for f in pre.families:
    proteins = glob.glob(os.path.join(pre.family_paths[f], "*.out"))
    print("Proteins for family %s" %f)
    for p in proteins:
        print(p)
    lengths[f] = len(p)
total = sum([lengths[f] for f in pre.families])

In [ ]:
padding = True
num_classes = 25
categorical = True
angles = False
normalize = False
max_length = 668
flatten = True

In [ ]:
batch_size = 64
intermediate_dim = 10
epochs = 20

### Build the autoencoder

In [ ]:
def create_checkpoints(f):
    checkpoints_path = os.path.join("models", f)
    tensorboard_path = os.path.join("logs", f)
    cp_cb = ModelCheckpoint(filepath=os.path.join(checkpoints_path, "model" + f + ".hdf5"), monitor='val_loss',
                            save_best_only=True)
    tb_cb = TensorBoard(log_dir=tensorboard_path)
    return [cp_cb, tb_cb]

In [ ]:
#autoencoder
def get_ae():
    if categorical:
        if not flatten:
            model_input = Input(shape=(None,num_classes))
        else:
            model_input = Input(shape=(max_length*num_classes,))
    else:
        model_input = Input(shape=(max_length,))
    #x=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(model_input)
    #encoded=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1, name="encoded")(x)
    #x=Conv1D(num_classes, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(encoded)
    encoded= Dense(intermediate_dim, activation='sigmoid')(model_input)
    if categorical:
        if not flatten:
            x = Dense(num_classes, activation='sigmoid')(encoded)
        else:
            x = Dense(max_length*num_classes, activation='sigmoid')(encoded)
    else:
        x = Dense(max_length, activation='sigmoid')(encoded)
    ae=Model(inputs=model_input, outputs=[x])
    opt=RMSprop(lr=0.01)
    ae.compile(optimizer=opt, loss='binary_crossentropy')
    ae.summary()
    return ae

### Train and evaluate the autoencoder for specific classes

In [ ]:
models = {"fam_1": "models/fam_1/modelfam_1.hdf5",
          "fam_2": "models/fam_2/modelfam_2.hdf5",
          "fam_3": "models/fam_3/modelfam_3.hdf5",
          "fam_4": "models/fam_4/modelfam_4.hdf5",
          "fam_5": "models/fam_5/modelfam_5.hdf5",
          "fam_6": "models/fam_6/modelfam_6.hdf5",
          "fam_7": "models/fam_7/modelfam_7.hdf5",
          "fam_8": "models/fam_8/modelfam_8.hdf5",
          "fam_9": "models/fam_9/modelfam_9.hdf5"}

In [ ]:
ds_serialized_path = "data_serialized"

In [ ]:
def evaluate_for_fam(f):
    print("Test for autoencoder on fam %s" %f)
    train_filename = os.path.join(ds_serialized_path, f, "train.npy")
    train = np.load(train_filename)
    ae = load_model(models[f])
    ae.summary()
    losses_train = []
    for t in train:
        losses_train.append(ae.evaluate(np.array([t]),np.array([t]), verbose=0))
    max_l = max(losses_train)
    losses_test = []
    del train
    tp, tn, fp, fn = 0, 0, 0, 0
    tp_p, tn_p, fp_p, fn_p = 0, 0, 0, 0
    for ft in families:
        print("Test for fam %s" %ft)
        test_filename = os.path.join(ds_serialized_path, ft, "test.npy")
        test = np.load(test_filename)
        for t in test:
            loss=ae.evaluate(np.array([t]),np.array([t]), verbose=0)
            if loss > max_l:
                # predict other family
                if ft == f:
                    fn+=1
                else:
                    tn+=1
            else:
                # predict current family
                if ft == f:
                    tp+=1 
                else:
                    fp+=1
            # compute the probability
            if loss > max_l:
                pr = 1 - (max_l / (2 * loss))
            else:
                pr = loss / (2 * max_l)
            if pr >= 0.5:
                # predict other family
                if ft == f:
                    fn_p+=1
                else:
                    tn_p+=1
            else:
                # predict current family
                if ft == f:
                    tp_p+=1 
                else:
                    fp_p+=1  
    return [tp, tn, fp, fn], [tp_p, tn_p, fp_p, fn_p]

In [ ]:
res_fam = open('res_fam_conf.csv', mode='w')
res_avg = open('res_avg_conf.csv', mode='w')
writer_fam = csv.writer(res_fam, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_fam.writerow(['Iteration', 'Superfamily', 'TP', 'TN', 'FP', 'FN', 'Prec', 'Recall', 'Spec', 'AUC'])
writer_avg = csv.writer(res_avg, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_avg.writerow(['Iteration', 'Prec', 'Recall', 'Spec', 'AUC'])
for i in range(0,20):
    for f in pre.families:
        families_conf = pre.load_family(f)
        families_conf = pre.process_conf(families_conf, categorical=categorical, angles=angles, padding=padding, max_length=max_length,normalize=normalize, flatten=flatten)  
        test_size = int(0.25 * families_conf.shape[0])
        val_size = int(0.15 * families_conf.shape[0])
        train_all, test = train_test_split(families_conf, test_size=test_size, random_state=i)
        train, val = train_test_split(train_all, test_size = val_size, random_state=i)
        del families_conf
        del train_all
        print("train: " + repr(train.shape))
        print("val: " + repr(val.shape))
        print("test: " + repr(test.shape))
        train_filename = os.path.join("data_serialized", f, "train.npy")
        val_filename = os.path.join("data_serialized", f, "val.npy")
        test_filename = os.path.join("data_serialized", f, "test.npy")
        np.save(train_filename, train)
        np.save(val_filename, val)
        np.save(test_filename, test)
        del train
        del test
        del val
    for f in pre.families:
        print("Training for family %s" %f)
        train_filename = os.path.join("data_serialized", f, "train.npy")
        test_filename = os.path.join("data_serialized", f, "val.npy")
        train = np.load(train_filename)
        test = np.load(test_filename)
        print("train: " + repr(train.shape))
        print("test" + repr(test.shape))
        ae = get_ae()
        ae.fit(train, train,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(test, test),
               callbacks=create_checkpoints(f),
               verbose=1)    
    # evaluate
    tp, tn, fp, fn = {}, {}, {}, {}
    tp_p, tn_p, fp_p, fn_p = {}, {}, {}, {}
    prec, recall, spec, auc = {}, {}, {}, {}
    prec_p, recall_p, spec_p, auc_p = {}, {}, {}, {}
    for f in pre.families:
        print("Evaluating family %s" %f)
        [tp[f], tn[f], fp[f], fn[f]], [tp_p[f], tn_p[f], fp_p[f], fn_p[f]] = evaluate_for_fam(f)
        prec_p[f] = (1.0* tp_p[f] / (tp_p[f] + fp_p[f]))
        recall_p[f] = (1.0* tp_p[f] / (tp_p[f] + fn_p[f]))
        spec_p[f] = (1.0* tn_p[f] / (tn_p[f] + fp_p[f]))
        auc_p[f] = (recall_p[f] + spec_p[f]) / 2
        # write to csv 
        writer_fam.writerow([i, f, tp_p[f], tn_p[f], fp_p[f], tn_p[f], prec_p[f], recall_p[f], spec_p[f], auc_p[f]])
    prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p = 0, 0, 0, 0

    for f in pre.families:
        prec_wavg_p += lengths[f] * prec_p[f] / total
        recall_wavg_p += lengths[f] * recall_p[f] / total
        spec_wavg_p += lengths[f] * spec_p[f] / total
        auc_wavg_p += lengths[f] * auc_p[f] / total
        # write to csv
        writer_avg.writerow([i, prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p])
res_fam.close()
res_avg.close()

In [ ]:
# import seaborn as sn
# import pandas as pd
# import matplotlib.pyplot as plt

# plt.figure(1, figsize=(15,15))
# for i,f in enumerate(families):
#     df_cm = pd.DataFrame([[tn[f], fp[f]],[ fn[f], tp[f]]], index = ["not " + f, f],
#                       columns = ["not " + f, f])
#     plt.subplot(330 + i + 1)
#     sn.heatmap(df_cm, annot=True, fmt='g')